# IFC Rasterization for Efficient Spatial Analysis

## Intro

### Import libraries and the IFC model:

In [77]:
import pyvista as pv
import numpy as np
from typing import Any, Tuple
import ifcopenshell
from importlib import reload 
import multiprocessing
import ifcopenshell
import ifcopenshell.geom
import time
import vtk
ifc_file = ifcopenshell.open(r"IFC Files\Project1.ifc")
#ifc_file = ifcopenshell.open(r"IFC Files\Duplex.ifc")

## Functions

In [78]:
def to_vtk_faces(faces : Tuple[tuple]) -> np.ndarray:
    faces=np.array(faces, dtype=np.int16)
    num_insertions = (len(faces) - 1) // 3
    # Generate an array of indices for insertions
    indices = np.arange(3, 3 * (num_insertions + 1), 3)
    indices = np.insert(indices, 0, 0)
    faces = np.insert(faces, indices, 3)
    return faces

def vtk_block_by_building_element(ifc_file):
    building_elements = ifc_file.by_type("IfcBuildingElement")
    settings = ifcopenshell.geom.settings()
    settings.set(settings.USE_WORLD_COORDS, True)
    settings.set(settings.APPLY_DEFAULT_MATERIALS, True)
    iterator = ifcopenshell.geom.iterator(settings, ifc_file, multiprocessing.cpu_count())
    multiblock = pv.MultiBlock()
    element_information = {} # Dictionary to hold element information
    exclude_list = ["IfcSpace", "IfcOpeningElement"]

    if iterator.initialize():
        while True:
            shape = iterator.get()
            if shape.type not in exclude_list:
                element = ifc_file.by_guid(shape.guid)
                           
                faces = shape.geometry.faces
                verts = shape.geometry.verts
                poly_data = pv.PolyData(list(verts), to_vtk_faces(faces))
                multiblock.append(poly_data)
                  
                if element in building_elements:
                #print(element.all_attributes()) --> why doesn't it work?
                        
                    element_information[shape.guid] = {
                    "Geo": poly_data, 
                    "Type": shape.type,
                    "Name": element.Name,
                    "Description": element.Description
                    }
                               
            if not iterator.next():
                break
                
    return multiblock, element_information

def create_uniform_grid(bounds, voxel_size):
    """Create a uniform grid within the given bounds."""
    x = np.arange(bounds[0], bounds[1] + voxel_size, voxel_size)
    y = np.arange(bounds[2], bounds[3] + voxel_size, voxel_size)
    z = np.arange(bounds[4], bounds[5] + voxel_size, voxel_size)
    return pv.StructuredGrid(*np.meshgrid(x, y, z))

def boxes_touch(A, B):
    """
    Check if two 3D bounding boxes touch or overlap.

    Parameters:
    - A, B: Tuples representing the bounds of boxes A and B.
      Each tuple should be in the format (xmin, xmax, ymin, ymax, zmin, zmax).
    
    Returns:
    - True if the boxes touch or overlap, otherwise False.
    """
    
    # Check for overlap in the x, y, and z dimensions
    overlap_x = A[0] <= B[1] and A[1] >= B[0]
    overlap_y = A[2] <= B[3] and A[3] >= B[2]
    overlap_z = A[4] <= B[5] and A[5] >= B[4]
    
    # Return True if all dimensions overlap, otherwise False
    return overlap_x and overlap_y and overlap_z

def compute_cell_index(grid, i, j, k):
    return i + j * (grid.dimensions[0]-1) + k *(grid.dimensions[0]-1)* (grid.dimensions[1]-1)

def find_cells_in_bounds(grid, mesh_bounds, voxel_size):
    """
    Find the indices of cells that are not entirely encapsulated by the mesh.

    Parameters:
    grid - The grid containing all the cells.
    mesh_bounds - The bounds of the mesh for which we want to find the cells that fall within.
    voxel_size - The size of each voxel along each axis.

    Returns:
    A list of indices of cells that are not entirely encapsulated by the mesh.
    """
    
    # Initialize an empty list to store the indices of cells that are not entirely encapsulated by the mesh
    cell_indices = []

    # Get the bounds of the entire grid
    grid_bounds = grid.bounds

    # Get the dimensions of the grid
    dims = grid.dimensions
    print(f'mesh bounds: {mesh_bounds}')
    print(f'voxel size: {voxel_size}')
    print(f'grid bounds: {grid_bounds}')
    print(f'grid dimensions: {dims}')
    
    # Calculate the starting and finishing cell indices based on the bounds of the mesh and the grid
    # Calculate the starting and finishing cell indices based on the bounds of the mesh and the grid

    starting_cell_index = [
        max(0, int(np.floor((mesh_bounds[0] - grid_bounds[0]) / voxel_size)) - 1),
        max(0, int(np.floor((mesh_bounds[2] - grid_bounds[2]) / voxel_size)) - 1),
        max(0, int(np.floor((mesh_bounds[4] - grid_bounds[4]) / voxel_size)) - 1)
    ]

    finishing_cell_index = [
        min(dims[0] - 1, int(np.ceil((mesh_bounds[1] - grid_bounds[0]) / voxel_size)) + 1),
        min(dims[1] - 1, int(np.ceil((mesh_bounds[3] - grid_bounds[2]) / voxel_size)) + 1),
        min(dims[2] - 1, int(np.ceil((mesh_bounds[5] - grid_bounds[4]) / voxel_size)) + 1)
    ]
    print(f'starting_cell_index: {starting_cell_index}')
    
    print(f'finishing_cell_index: {finishing_cell_index}')
    
    # Iterate through all cells in the grid from the starting to the finishing cell index
    # and add the indices of cells that are not entirely encapsulated by the mesh to the list
    for i in range(starting_cell_index[0], finishing_cell_index[0] + 1):
        for j in range(starting_cell_index[1], finishing_cell_index[1] + 1):
            for k in range(starting_cell_index[2], finishing_cell_index[2] + 1):
                # Calculate the cell index from the i, j, and k indices
                #print(f'i, j, k: {i}, {j}, {k}')
                cell_index = compute_cell_index(grid, i, j, k)  
                #print(f'cell index: {cell_index}')
                cell_indices.append(cell_index)
    #print(cell_indices)
    return cell_indices



In [79]:
def voxelize_space(meshes, mesh_info):
    """Voxelize space and check intersections with given mesh."""
    start_time = time.time()
    voxel_size = 0.8
    # Step 1: Create a uniform grid within the bounds of all meshes
    grid = create_uniform_grid(meshes.bounds, voxel_size)
    
    end_time = time.time()
    print(f"Time taken to create the initial grid: {end_time - start_time:.4f} seconds")

    num_points = grid.cell_centers().n_points
    mask = np.zeros(num_points, dtype=bool)
    cell_to_guid = {}

    start_time = time.time()

    print(f'number of cells: {num_points}')
    
    # Step 2: Iterate through the meshes in the scene
    for guid, mesh_data in mesh_info.items():
        
        mesh = mesh_data['Geo']
        mesh_bounds = mesh.bounds
        
        # Find the indices of the cells that fall into the bounds of the current mesh
        cell_indices = find_cells_in_bounds(grid, mesh_bounds, voxel_size)

        cell_indices.sort()
        print(f'cell indices: {cell_indices}')

        # Step 3: Use the cell indices to check which cells need further proofing
        for cell_index in cell_indices:
            cell = grid.extract_cells([cell_index])
            cell_bounds = cell.bounds

            if boxes_touch(mesh_bounds, cell_bounds):
                if not mask[cell_index]:
                    mask[cell_index] = True
                    cell_to_guid[cell_index] = guid

    end_time = time.time()

    print(f"Time taken to voxelize: {end_time - start_time:.4f} seconds")        
                            
    return grid, mask, cell_to_guid


## Voxelization

In [80]:
p = pv.Plotter()

start_time = time.time()
all_meshes, info = vtk_block_by_building_element(ifc_file)
end_time = time.time()
print(f"Time taken to import ifc objects: {start_time-end_time:.4f} seconds")

# Voxelize the entire space of the combined mesh
grid, mask, cell_info = voxelize_space(all_meshes, info)
xx = []
for x in range(len(mask)):
        if mask[x] == 1:
            xx.append(x)
print(f'The indices that will be colored red with the mask: {xx}')
# Visualization
p.add_mesh(grid, opacity=0.3, show_edges=True)
print('end')
p.add_mesh(grid.extract_cells(np.where(mask)[0]), color="red", opacity=0.5)
p.add_point_labels(grid.cell_centers(), labels=[f"{i}" for i in range(grid.cell_centers().n_points)])

p.show()


Time taken to import ifc objects: -0.1166 seconds
Time taken to create the initial grid: 0.0000 seconds
number of cells: 180
mesh bounds: (-2.306903294088416, 2.293096705911564, -2.659084903543768, 1.953415096456222, -0.48, 0.0)
voxel size: 0.8
grid bounds: (-2.3069032940884404, 2.4930967059115603, -2.679084903543776, 2.120915096456225, -0.48, 3.52)
grid dimensions: (7, 7, 6)
starting_cell_index: [0, 0, 0]
finishing_cell_index: [6, 6, 2]
cell indices: [0, 1, 2, 3, 4, 5, 6, 6, 7, 8, 9, 10, 11, 12, 12, 13, 14, 15, 16, 17, 18, 18, 19, 20, 21, 22, 23, 24, 24, 25, 26, 27, 28, 29, 30, 30, 31, 32, 33, 34, 35, 36, 36, 36, 37, 37, 38, 38, 39, 39, 40, 40, 41, 41, 42, 42, 42, 43, 44, 45, 46, 47, 48, 48, 49, 50, 51, 52, 53, 54, 54, 55, 56, 57, 58, 59, 60, 60, 61, 62, 63, 64, 65, 66, 66, 67, 68, 69, 70, 71, 72, 72, 72, 73, 73, 74, 74, 75, 75, 76, 76, 77, 77, 78, 78, 78, 79, 80, 81, 82, 83, 84, 84, 85, 86, 87, 88, 89, 90, 90, 91, 92, 93, 94, 95, 96, 96, 97, 98, 99, 100, 101, 102, 102, 103, 104, 105,

Widget(value="<iframe src='http://localhost:49946/index.html?ui=P_0x159004fe9b0_19&reconnect=auto' style='widt…